In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf



In [2]:
mp_hands = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Data') 

# Actions that we try to detect
actions = np.array(['ban dang lam gi', 'ban di dau the', 'ban hieu ngon ngu ky hieu khong', 'ban hoc lop may',
                    'ban khoe khong', 'ban muon gio roi', 'ban phai canh giac', 'ban ten la gi',
                    'ban tien bo day', 'ban trong cau co the', 'bo me toi cung la nguoi Diec', 
                    'cai nay bao nhieu tien', 'cai nay la cai gi', 'cam on', 'cap cuu', 'chuc mung', 'con yeu me',
                    'cong viec cua ban la gi', 'hen gap lai cac ban', 'mon nay khong ngon', 'toi bi chong mat',
                    'toi bi cuop', 'toi bi dau dau', 'toi bi dau hong', 'toi bi ket xe', 'toi bi lac',
                    'toi cam thay rat hoi hop', 'toi cam thay rat vui', 'toi can an sang', 'toi can di ve sinh', 
                    'toi can gap bac si', 'toi can phien dich', 'toi dang an sang', 'toi dang buon', 
                    'toi dang o ben xe', 'toi dang o cong vien', 'toi dang phai cach ly', 'toi di sieu thi', 
                    'toi di toi Ha Noi', 'toi khoi benh roi', 'toi khong hieu', 'toi khong quan tam', 
                    'toi la hoc sinh', 'toi la nguoi Diec', 'toi song o Ha Noi', 'toi thay doi bung', 
                    'toi thay nho ban', 'toi thich an mi', 'toi thich phim truyen', 'xin chao'])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 60

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [9]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'ban dang lam gi': 0,
 'ban di dau the': 1,
 'ban hieu ngon ngu ky hieu khong': 2,
 'ban hoc lop may': 3,
 'ban khoe khong': 4,
 'ban muon gio roi': 5,
 'ban phai canh giac': 6,
 'ban ten la gi': 7,
 'ban tien bo day': 8,
 'ban trong cau co the': 9,
 'bo me toi cung la nguoi Diec': 10,
 'cai nay bao nhieu tien': 11,
 'cai nay la cai gi': 12,
 'cam on': 13,
 'cap cuu': 14,
 'chuc mung': 15,
 'con yeu me': 16,
 'cong viec cua ban la gi': 17,
 'hen gap lai cac ban': 18,
 'mon nay khong ngon': 19,
 'toi bi chong mat': 20,
 'toi bi cuop': 21,
 'toi bi dau dau': 22,
 'toi bi dau hong': 23,
 'toi bi ket xe': 24,
 'toi bi lac': 25,
 'toi cam thay rat hoi hop': 26,
 'toi cam thay rat vui': 27,
 'toi can an sang': 28,
 'toi can di ve sinh': 29,
 'toi can gap bac si': 30,
 'toi can phien dich': 31,
 'toi dang an sang': 32,
 'toi dang buon': 33,
 'toi dang o ben xe': 34,
 'toi dang o cong vien': 35,
 'toi dang phai cach ly': 36,
 'toi di sieu thi': 37,
 'toi di toi Ha Noi': 38,
 'toi khoi benh ro

In [10]:
sequences, labels = [], []
for action in actions:b
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
# cái này chạy hơi lâu (tầm 15-30p) nên a có thể tranh thủ làm việc khác

In [11]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [11]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(60,126))) # 32
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [13]:
model.load_weights('model 50 câu 95,33/95_sc_50.h5') # main

In [17]:
import cv2   
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0) 
# Set mediapipe model 
with mp_hands.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read() # đọc video từ camera

        # Make detections
        image, results = mediapipe_detection(frame, holistic) # gọi biến image và result = giá trị RGB của ảnh
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results) # nhận diện cơ thể 
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results) # xuất ra array các keypoints cơ thể (mấy cái đốt í)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-60:] # Đưa các frame thu đc vào mảng sequence, ở đây t để là 30 frame cuối
        
        if len(sequence) == 60:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)]) # Nếu thu đủ số frame của câu, đưa các array đấy vào model rồi ước lượng ra kết quả chính xác nhất trong dữ liệu
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            # image = prob_viz(res, actions, image, colors)
        
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        if len(sequence) == 60:
            cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti